In [3]:
import matplotlib.pyplot as plt
import numpy as np

import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("cbc") or os.path.isfile("cbc")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq coinor-cbc
    else:
        try:
            !conda install -c conda-forge coincbc 
        except:
            pass


assert(shutil.which("cbc") or os.path.isfile("cbc"))
from pyutilib.services import register_executable, registered_executable
register_executable(name='glpsol')
!apt-get install -y -qq glpk-utils
!apt-get install -y -qq coinor-cbc
from pyomo.environ import *

In [ ]:
from pyomo.environ import *

MACHINES = ['A', 'B', 'C', 'D']
PARTS = ['P1', 'P2', 'P3', 'P4']

max_avail_hours = {'A': 60, 'B': 60, 'C': 50, 'D': 40}
time_needed = {
    ('A', 'P1'): 30, ('A', 'P2'): 20, ('A', 'P3'): 1000, ('A', 'P4'): 40,
    ('B', 'P1'): 30, ('B', 'P2'): 30, ('B', 'P3'): 30, ('B', 'P4'): 1000,
    ('C', 'P1'): 25, ('C', 'P2'): 25, ('C', 'P3'): 25, ('C', 'P4'): 25,
    ('D', 'P1'): 1000, ('D', 'P2'): 1000, ('D', 'P3'): 40, ('D', 'P4'): 40
}
cost_per_hour = {'A': 300, 'B': 250, 'C': 350, 'D': 200}

opt_model = ConcreteModel()

opt_model.y = Var(MACHINES, PARTS, within=NonNegativeReals, initialize=0)

# Define objective function
opt_model.total_cost = Objective(expr=sum(cost_per_hour[m]*sum(opt_model.y[m, p]*time_needed[m, p] for p in PARTS) for m in MACHINES), sense=minimize)

# Define constraints
opt_model.machine_cons = ConstraintList()
for m in MACHINES:
    opt_model.machine_cons.add(sum(time_needed[m, p]*opt_model.y[m, p] for p in PARTS) <= max_avail_hours[m])

opt_model.part_cons = ConstraintList()
for p in PARTS:
    opt_model.part_cons.add(sum(opt_model.y[m, p] for m in MACHINES) == 1)

# Solve model
solver_opt = SolverFactory('glpk')
solver_opt.solve(opt_model)

# Print results
for m in MACHINES:
    for p in PARTS:
        print(f"y[{m}, {p}] = {opt_model.y[m, p].value}")

print(f"Minimum cost = {opt_model.total_cost()}")
